## Setup

Downloads the FHIR Validator from https://github.com/hapifhir/org.hl7.fhir.core/releases/latest/download/validator_cli.jar

For more details on the FHIR Validator see [Using the FHIR Validator](https://confluence.hl7.org/spaces/FHIR/pages/35718580/Using+the+FHIR+Validator)

Downloads the NW Genomics validation package

In [33]:
import urllib.request
from tqdm import tqdm
import requests

print("Downloading the FHIR Validator...")

url = "https://github.com/hapifhir/org.hl7.fhir.core/releases/latest/download/validator_cli.jar"
response = requests.get(url, stream=True)

with open("validator_cli.jar", "wb") as handle:
    for data in tqdm(response.iter_content(chunk_size=1024), unit="kB"):
        handle.write(data)

print("Downloading the North West Genomics validation package...")
url = "https://nw-gmsa.github.io/package.tgz"
response = requests.get(url, stream=True)

with open("package.tgz", "wb") as handle:
    for data in tqdm(response.iter_content(chunk_size=1024), unit="kB"):
        handle.write(data)


Now run a validation using a FHIR Message Bundle.

The example is an O21 Order, so we also need to help the validator by specifying the profile of the focus resource which in this case is a ServiceRequest which should adhere to https://nw-gmsa.github.io/StructureDefinition-ServiceRequest.html (profile url https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest)

A default output is returned from the validator.

Using command line this is:

>  java -jar validator_cli.jar Input/FHIR/O21/Liverpool_O21.json -version 4.0.1 -ig package.tgz -bundle ServiceRequest:0 https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest -tx n/a

In [34]:
import subprocess

filename = "Liverpool_O21.json"
folder = 'FHIR/O21/'

subprocess.run(["java", "-jar", "validator_cli.jar", "Input/"+folder + filename , "-version", "4.0.1", "-ig", "package.tgz", "-bundle", "ServiceRequest:0", "https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest", "-tx", "n/a"])

CompletedProcess(args=['java', '-jar', 'validator_cli.jar', 'Input/FHIR/O21/Liverpool_O21.json', '-version', '4.0.1', '-ig', 'package.tgz', '-bundle', 'ServiceRequest:0', 'https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest', '-tx', 'n/a'], returncode=0)

The output can contain a mix of issues which may be difficult to understand. If we get a large number of issues, we may want to prioritise `error` first and write a test report. So we need to output the results in a machine readable format



In [35]:
import subprocess

subprocess.run(["java", "-jar", "validator_cli.jar", "Input/"+folder + filename, "-version", "4.0.1", "-ig", "package.tgz", "-bundle", "ServiceRequest:0", "https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest", "-tx", "n/a", "-output", "Results/"+folder + filename + "-OperationOutcome.json", "-output-style","json"])

CompletedProcess(args=['java', '-jar', 'validator_cli.jar', 'Input/FHIR/O21/Liverpool_O21.json', '-version', '4.0.1', '-ig', 'package.tgz', '-bundle', 'ServiceRequest:0', 'https://fhir.nwgenomics.nhs.uk/StructureDefinition/ServiceRequest', '-tx', 'n/a', '-output', 'Results/FHIR/O21/Liverpool_O21.json-OperationOutcome.json', '-output-style', 'json'], returncode=0)

In [36]:
import json
import pandas as pd

def details(item):
    return item['text']

with open( "Results/"+folder + filename + "-OperationOutcome.json", 'rb') as f:
    s = f.read().decode('utf-8')
    operationOutcome = json.loads(s)

df = pd.DataFrame(operationOutcome['issue'])

df['details'] = df['details'].apply(details)
df.drop(columns=['extension'], inplace=True)
df.sort_values(by=['severity'], inplace=True)

df

,severity,code,details,expression
0,error,invalid,"The property identifier must be an Object, not...",[Bundle.entry[2].resource.identifier[0].assign...
16,error,structure,"Encounter.class: minimum required = 1, but onl...",[Bundle.entry[3].resource/*Encounter/3b5120c0-...
14,error,invalid,No definition could be found for URL value 'ht...,[Bundle.entry[2].resource/*Patient/dc969479-f3...
145,error,structure,Unable to find a profile match for urn:uuid:3b...,[Bundle.entry[1].resource.encounter]
11,error,invariant,Constraint failed: cpt-2: 'A system is require...,[Bundle.entry[2].resource/*Patient/dc969479-f3...
...,...,...,...,...
83,warning,invalid,Coding has no system. A code with no system ha...,[Bundle.entry[15].resource/*Observation/OBX-7-...
82,warning,invalid,"Best Practice Recommendation: In general, all ...",[Bundle.entry[15].resource/*Observation/OBX-7-...
81,warning,invalid,"Best Practice Recommendation: In general, all ...",[Bundle.entry[15].resource/*Observation/OBX-7-...
89,warning,invalid,Coding has no system. A code with no system ha...,[Bundle.entry[16].resource/*Observation/OBX-4-...


In [37]:
import jinja2
##from xhtml2pdf import cairo
from datetime import date

html_string = df.to_html()

with open("Results/"+folder + filename + "-report.html", "w") as out_html_file_handle:
    out_html_file_handle.write(html_string)

#html = (
#    jinja2.Environment(loader=jinja2.FileSystemLoader(searchpath=""))
#    .get_template(name="report_template.html")
#    .render(
#        date=date.today().strftime("%d, %b %Y"),
#        row_ix=1,
#        row=df,
#    )
#)


    # Convert HTML to PDF
    #with open("report_row_%s.pdf" % (row_ix + 1), "w+b") as out_pdf_file_handle:
    #    pisa.CreatePDF(
    #        # HTML to convert
    #        src=html,
    #        # File handle to receive the result
    #        dest=out_pdf_file_handle,
    #    )
